In [10]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.10.1'

In [11]:
model = keras.models.load_model('bees-wasps.h5', compile=False)
model.compile()

# Question 1

In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('bees-wasps.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\p114349\AppData\Local\Temp\tmpy2znrueb\assets


INFO:tensorflow:Assets written to: C:\Users\p114349\AppData\Local\Temp\tmpy2znrueb\assets


# Question 2

In [14]:
import tensorflow.lite as tflite

In [17]:
interpreter = tflite.Interpreter(model_path='bees-wasps.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
output_index

13

# Question 3

In [18]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [24]:
def preprocess_input(x):
    x /= 255
    return x

In [33]:
img = download_image('https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg')
img = prepare_image(img, (150,150))

x = np.array(img, dtype='float32')
X = np.array([x])


X = preprocess_input(X)
X[0,0,0,:]

array([0.94509804, 0.90588236, 0.85882354], dtype=float32)

# Question 4

In [34]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [41]:
preds[0][0]

0.65898407

# Question 6

In [42]:
import requests

url = 'http://localhost:8080/2015-03-31/functions/function/invocations'

data = {'url': 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'}

result = requests.post(url, json=data).json()
print(result)

{'errorMessage': 'Unable to marshal response: Object of type float32 is not JSON serializable', 'errorType': 'Runtime.MarshalError', 'requestId': 'bffa6025-6030-4734-91af-8766ba057989', 'stackTrace': []}
